In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://host-192-168-2-218-de1:7077") \
        .appName("Jakob_Haggstrom_test")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",1)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 11:09:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/05 11:09:50 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:
# Read all the data
data = spark_session.read.option("recursiveFileLookup", "true").json('hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/B/A')

# Read a small subset of data:
#data = spark_session.read.json('hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/B/A/A/*.json')

In [3]:
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- similars: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- tags: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- track_id: string (nullable = true)



In [16]:
from pyspark.sql.functions import flatten
from pyspark.sql.types import ArrayType, StringType

data_new1 = data.withColumn("tags_new", flatten(data.tags))

data_new1.select("tags_new").first()




Row(tags_new=['pop', '100', 'rock', '83', 'maroon 5', '78', 'alternative', '59', 'pop rock', '51', 'Love', '27', 'alternative rock', '18', 'she will be loved', '16', 'Ballad', '12', 'Mellow', '10', 'male vocalists', '10', '00s', '9', 'soft rock', '8', 'favorites', '8', 'romantic', '7', 'Maroon5', '7', 'chill', '7', 'american', '6', 'love songs', '6', 'indie', '6', 'beautiful', '6', 'chillout', '5', 'memories', '5', 'sad', '4', 'Pop-Rock', '3', 'acoustic', '3', 'Favorite', '3', 'love song', '3', 'Maroon 5 - She will be loved', '3', 'easy listening', '3', 'relax', '2', 'male vocalist', '2', 'Favourites', '2', 'favorite songs', '2', 'Maroon 5 She Will Be Loved', '2', 'funk', '2', '2004', '2', 'indie rock', '2', 'Sentimental', '2', 'loved', '1', 'slow', '1', 'One Tree Hill', '1', 'sweet', '1', 'ballads', '1', 'nostalgic', '1', 'catchy', '1', 'emo', '1', 'melancholic', '1', 'soft', '1', 'amazing', '1', 'cool', '1', 'Awesome', '1', 'top 40', '1', 'Good Stuff', '1', 'favourite', '1', 'guilty 

In [21]:
data_new1.printSchema()

root
 |-- artist: string (nullable = true)
 |-- similars: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- tags: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- tags_new: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [22]:
from pyspark.sql.functions import udf, col

map_func_1 = lambda x: x[0::2]
udf_1 = udf(map_func_1, ArrayType(StringType()))

data_new1 = data_new1.withColumn("tag_names", udf_1(col("tags_new")))

data_new1.select("tag_names").first()

Row(tag_names=['pop', 'rock', 'maroon 5', 'alternative', 'pop rock', 'Love', 'alternative rock', 'she will be loved', 'Ballad', 'Mellow', 'male vocalists', '00s', 'soft rock', 'favorites', 'romantic', 'Maroon5', 'chill', 'american', 'love songs', 'indie', 'beautiful', 'chillout', 'memories', 'sad', 'Pop-Rock', 'acoustic', 'Favorite', 'love song', 'Maroon 5 - She will be loved', 'easy listening', 'relax', 'male vocalist', 'Favourites', 'favorite songs', 'Maroon 5 She Will Be Loved', 'funk', '2004', 'indie rock', 'Sentimental', 'loved', 'slow', 'One Tree Hill', 'sweet', 'ballads', 'nostalgic', 'catchy', 'emo', 'melancholic', 'soft', 'amazing', 'cool', 'Awesome', 'top 40', 'Good Stuff', 'favourite', 'guilty pleasure', 'soul', 'lovely', 'West Coast swing', 'singer-songwriter', 'USA', 'my youth', 'maroon', 'fav', '2000s', 'breaks my heart', 'Guilty Pleasures', 'good', 'Love it', 'falling asleep sadly', 'melodic', 'expressive', 'america', 'poprock', 'adult contemporary', 'Verbotene Liebe', '

In [30]:
data_new1.printSchema()


root
 |-- artist: string (nullable = true)
 |-- similars: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- tags: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- tags_new: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tag_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Gender tag: string (nullable = true)
 |-- Mod_gender_tags: string (nullable = true)



In [34]:
def get_gender_tag(lst): 
        female_lst = ['female vocalist', 'female vocalists']
        male_lst = ['male vocalist', 'male vocalists']
        mixed_list = [['male vocalist', 'female vocalists'], ['male vocalists', 'female vocalist'], ['male vocalists', 'female vocalists']]

        for sub_list in mixed_list: # if any of the combinations in mixed_list occur, then it is mixed
                if all(gender in lst for gender in sub_list):
                        return 'mixed'
        
        if any(gender in lst for gender in female_lst): # Grouping together single and multiple vocalists.
                return 'female'
        if any(gender in lst for gender in male_lst):
                return 'male'
        
        return 'unknown'

gender_tag = udf(get_gender_tag, StringType())

data_new1 = data_new1.withColumn("Gender tag", gender_tag(col("tag_names")))

data_new1.select("Gender tag").take(10)
        


[Row(Gender tag='male'),
 Row(Gender tag='female'),
 Row(Gender tag='unknown'),
 Row(Gender tag='male'),
 Row(Gender tag='male'),
 Row(Gender tag='mixed'),
 Row(Gender tag='male'),
 Row(Gender tag='female'),
 Row(Gender tag='female'),
 Row(Gender tag='female')]

In [46]:
from pyspark.sql.functions import array_remove

gender_tags = ['female vocalist', 'female vocalists', 'male vocalist', 'male vocalists']

for gender in gender_tags: # Removing the tags that labelled the gender of the vocalist
        data_new1 = data_new1.withColumn("Mod_gender_tags", array_remove(data_new1.Mod_gender_tags, gender))



data_new1.select("Mod_gender_tags").take(10)

[Row(Mod_gender_tags=['pop', 'rock', 'maroon 5', 'alternative', 'pop rock', 'Love', 'alternative rock', 'she will be loved', 'Ballad', 'Mellow', '00s', 'soft rock', 'favorites', 'romantic', 'Maroon5', 'chill', 'american', 'love songs', 'indie', 'beautiful', 'chillout', 'memories', 'sad', 'Pop-Rock', 'acoustic', 'Favorite', 'love song', 'Maroon 5 - She will be loved', 'easy listening', 'relax', 'Favourites', 'favorite songs', 'Maroon 5 She Will Be Loved', 'funk', '2004', 'indie rock', 'Sentimental', 'loved', 'slow', 'One Tree Hill', 'sweet', 'ballads', 'nostalgic', 'catchy', 'emo', 'melancholic', 'soft', 'amazing', 'cool', 'Awesome', 'top 40', 'Good Stuff', 'favourite', 'guilty pleasure', 'soul', 'lovely', 'West Coast swing', 'singer-songwriter', 'USA', 'my youth', 'maroon', 'fav', '2000s', 'breaks my heart', 'Guilty Pleasures', 'good', 'Love it', 'falling asleep sadly', 'melodic', 'expressive', 'america', 'poprock', 'adult contemporary', 'Verbotene Liebe', 'The 4400', 'new', 'sadness',

In [53]:
final_df = data_new1.select("Gender tag", "Mod_gender_tags")

final_df.printSchema()

root
 |-- Gender tag: string (nullable = true)
 |-- Mod_gender_tags: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [54]:
from pyspark.sql.functions import explode

final_df = final_df.select(final_df["Gender tag"], explode(final_df["Mod_gender_tags"]))


In [55]:
final_df.printSchema()

root
 |-- Gender tag: string (nullable = true)
 |-- col: string (nullable = true)



In [56]:
final_df = final_df.withColumnRenamed("col", "Tags")

In [59]:
# These last lines does essentially the same thing. They count the frequency of each tag for given gender. 
tag_freq_df_female = final_df.select("Tags").filter(final_df["Gender tag"] == "female").groupBy("Tags").agg({"Tags": "count"})\
        .withColumnRenamed("count(Tags)","Frequency of tag").orderBy("Frequency of tag", ascending=False)

tag_freq_df_female.show()

22/03/05 14:34:00 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.2.152:38156 is closed


+-----------------+----------------+
|             Tags|Frequency of tag|
+-----------------+----------------+
|              pop|              36|
|             rock|              28|
|              00s|              19|
|             Love|              19|
|            dance|              18|
|           female|              18|
|singer-songwriter|              17|
|             soul|              15|
|       electronic|              15|
|         american|              13|
|        beautiful|              13|
|         chillout|              12|
|      alternative|              12|
|             jazz|              10|
|              80s|              10|
|              rnb|               9|
|             sexy|               9|
|        favorites|               9|
|             folk|               9|
|            indie|               9|
+-----------------+----------------+
only showing top 20 rows



In [60]:
tag_freq_df_male = final_df.select("Tags").filter(final_df["Gender tag"] == "male").groupBy("Tags").agg({"Tags": "count"})\
        .withColumnRenamed("count(Tags)","Frequency of tag").orderBy("Frequency of tag", ascending=False)

tag_freq_df_male.show()

+-----------------+----------------+
|             Tags|Frequency of tag|
+-----------------+----------------+
|             rock|              30|
|              pop|              24|
|             Love|              20|
|        favorites|              20|
|      alternative|              18|
|          Awesome|              17|
|        beautiful|              15|
|            indie|              14|
|              00s|              12|
| alternative rock|              12|
|     classic rock|              12|
|         pop rock|              11|
|singer-songwriter|              11|
|          british|              10|
|           catchy|              10|
|         american|              10|
|              sad|              10|
|           oldies|              10|
|           Ballad|               9|
|             cool|               9|
+-----------------+----------------+
only showing top 20 rows



In [61]:
tag_freq_df_mixed = final_df.select("Tags").filter(final_df["Gender tag"] == "mixed").groupBy("Tags").agg({"Tags": "count"})\
        .withColumnRenamed("count(Tags)","Frequency of tag").orderBy("Frequency of tag", ascending=False)


+-----------------+----------------+
|             Tags|Frequency of tag|
+-----------------+----------------+
|             rock|              30|
|              pop|              24|
|             Love|              20|
|        favorites|              20|
|      alternative|              18|
|          Awesome|              17|
|        beautiful|              15|
|            indie|              14|
|              00s|              12|
| alternative rock|              12|
|     classic rock|              12|
|         pop rock|              11|
|singer-songwriter|              11|
|         american|              10|
|          british|              10|
|           catchy|              10|
|              sad|              10|
|           oldies|              10|
|           Ballad|               9|
|             cool|               9|
+-----------------+----------------+
only showing top 20 rows



In [62]:
tag_freq_df_mixed.show()

+--------------------+----------------+
|                Tags|Frequency of tag|
+--------------------+----------------+
|          Soundtrack|               2|
|                 pop|               2|
|                rock|               2|
|           beautiful|               2|
|               party|               2|
|           nostalgic|               2|
|      easy listening|               2|
|           favorites|               2|
|               dance|               2|
|                duet|               2|
|               happy|               2|
|                Love|               2|
|          last dance|               1|
|           sentyment|               1|
|Music From Chick ...|               1|
|            Pop Life|               1|
|        feeling good|               1|
|            romantic|               1|
|            adoroooo|               1|
|         tnphp-dec27|               1|
+--------------------+----------------+
only showing top 20 rows



In [63]:
spark_context.stop()